# Pronova LLM V1 **(Outdated now. Use new "Run" and "Train" notebooks)** #
## Use this notebook to do the following ##
- Create Qdrant collections
- Chunk text files into smaller chunks
- Create embeddings for data chunks and querys
- Delete qdrant vectors
- Run the current model on a query

In [1]:
# Load require librarys
import os
from qdrant_client import QdrantClient
from qdrant_client.http import models
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

# Load environment variables from .env file
load_dotenv()

/Users/irisyang1010/Documents/Capstone/pronova_dog_LLM/.venv/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


True

### Setup Qdrant connection ###

In [2]:
# Get the Qdrant API key from the environment variable
Qdrant_api_key = os.getenv('Qdrant_API_KEY')
if not Qdrant_api_key:
    raise ValueError("No Qdrant API key found in environment variables")
Qdrant_url = os.getenv('Qdrant_URL')
if not Qdrant_url:
    raise ValueError("No Qdrant URL found in environment variables")


# Initialize Qdrant client
try:
    Qclient = QdrantClient(
        url= Qdrant_url,
        api_key=Qdrant_api_key
    )
    print("Successfully connected to Qdrant")
except Exception as e:
    print(f"Failed to connect to Qdrant: {e}")
    raise

Successfully connected to Qdrant


### Setup OpenAI connection ###

In [3]:
# Get the OpenAI API key from the environment variable
OpenAI_api_key = os.getenv('OPENAI_API_KEY')
if not OpenAI_api_key:
    raise ValueError("No OpenAI API key found in environment variables")

OpenAI.api_key = OpenAI_api_key

### Creating a Qdrant Collection (function) ###

In [4]:
#Create collection
def create_qdrant_collection(collection_name):
    try:
        Qclient.create_collection(
            collection_name=collection_name,
            vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
        )
        print(f"Collection '{collection_name}' created successfully")
    except Exception as e:
        print(f"Failed to create collection '{collection_name}': {e}")
        raise

### Get an OpenAI embedding from a text segment (Function) ###

In [5]:
# Function to get the embedding of a text
def get_embedding(text):
    client = OpenAI()
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response.data[0].embedding

### Turn a file into chunks (Function) ###

In [6]:
# Function to read a text file and chunk its content
def chunk_text_from_file(file_path, chunk_size=400):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

### Get embedding from a list of chunks (Function) ###

In [7]:
# Function to get embeddings for a list of text chunks
def get_embeddings_for_chunks(chunks):
    embeddings = []
    for chunk in chunks:
        embedding = get_embedding(chunk)
        embeddings.append(embedding)
    return embeddings

### Get largest Qdrant ID (function) ###

In [8]:
def get_qdrant_next_id(collection_name):
    try:
        collection = Qclient.get_collection(collection_name=collection_name)
        id = collection.points_count
        if id == None:
            # print("id was zero")
            return 0
        else:
            # print(f"Next ID in collection '{collection_name}' is {id}")
            return id
    except Exception as e:
        print(f"Failed to get next ID from collection '{collection_name}': {e}")
        raise

### Upsert lists of documents to Qdrant (Function) ###
#### Mind the parameters ####

In [9]:
# function to upsert embeddings into Qdrant
# def upsert_embeddings(collection_name, embeddings, chunks):
def upsert_embeddings(collection_name, embeddings, chunks, filename):
    largest_id = get_qdrant_next_id(collection_name)
    points = []
    for i in range(len(embeddings)):
        points.append(
            {
                "id": i + largest_id + 1,
                "vector": embeddings[i],
                "payload": {
                    "text": chunks[i],   # Attach the chunk as payload
                    "source_file": filename  # Add source file
                }            
            }
        )

    try:
        Qclient.upsert(
            collection_name=collection_name,
            points=points
        )
        print("Embeddings upserted successfully")
    except Exception as e:
        print(f"Failed to upsert embeddings: {e}")
        raise

### Retrieve similar chunks from query (Function) ###

In [10]:
def retrieve_relevant_chunks(collection_name, query, top_k=10):
    query_embedding = get_embedding(query)
    
    search_result = Qclient.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=top_k
    )

    contexts = [result.payload["text"] for result in search_result]
    files = [result.payload.get("source_file") for result in search_result]
    
    return contexts, files


### Generate Response from Query (Function) ###

In [11]:
from collections import Counter

def file_ratios(files):
    total_files = len(files)
    counts = Counter(files)
    return {file: count*100 / total_files for file, count in counts.items()}


# file_ratios(["a", "a", "b", "c"])
# {'a': 0.5, 'b': 0.25, 'c': 0.25}

In [12]:
import numpy as np

def generate_response(collection_name, query):
    context, files = retrieve_relevant_chunks(collection_name, query)
    
    # unique_files = np.unique(files)
    file_rank = file_ratios(files)
    

    system_role = "You are a specialized assistant that only provides advice on dog-related veterinary care. If a user asks about any other animal or topic outside of dog health, politely decline to answer and remind them that you only provide information about dogs."

    # If you don't know the answer to a question, let the user know that you're not sure and suggest that they consult a veterinarian for more information.

    # Combine retrieved chunks into a single string
    context_text = "\n".join(context)


    # Generate a response using GPT-4
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": system_role},
            {"role": "user", "content": "context" + context_text},
            {"role": "user", "content": query}
        ]
    )
    return completion.choices[0].message, file_rank

### Delete all entries in a collection (Function) ###
*BE CAREFUL WITH THIS*

In [ ]:
def reset_model(collection_name):
    confirmation = input("Type 'DELETE' to confirm deletion of entries in " + collection_name + ". This cannot be undone. Type anything else to abort")
    if confirmation == "DELETE" :
        try:
            Qclient.delete_collection(collection_name=collection_name)
            print(f"Collection '{collection_name}' deleted successfully")
            create_qdrant_collection(collection_name)
        except Exception as e:
            print(f"Failed to delete collection '{collection_name}': {e}")
            raise
    else:
        print("Deletion aborted")

In [15]:
# Qclient.get_collections() 

### Markdown Print Function ###

In [13]:
def print_markdown(md_text):
    display(Markdown(md_text))

### Process Files From Folder (function) ###

In [14]:
def process_files_from_folder(collection_name, file_path, limit):
    
    #number of files in the folder 
    num_files = len([name for name in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, name))])
    if limit == None:
        limit = num_files
    current_file = 1

    for filename in os.listdir(file_path):
        if current_file > limit:
            break
        curr_file_path = os.path.join(file_path, filename)
        print(f"Processing file {current_file} of max {limit}: {filename}")
        
        # # Chunk the file
        chunks = chunk_text_from_file(curr_file_path)
        # Get embeddings for the chunks
        embeddings = get_embeddings_for_chunks(chunks)
        # Upsert the embeddings into Qdrant
        upsert_embeddings(collection_name, embeddings, chunks, filename)
        current_file += 1


## Playground ##
Use this section to test functions and play around with the model

In [17]:

# delete_collection("LLM_V1")
filepath = "scrapingDemo/ScrapedFiles_petMD_behavior"
collection_name = "LLM_V1"
limit = None
# delete_collection(collection_name)
process_files_from_folder(collection_name, filepath, limit)

Processing file 1 of max 14: Why_Is_My_Dog_Shaking.txt
Failed to get next ID from collection 'LLM_V1': Unexpected Response: 404 (Not Found)
Raw response content:
b'404 page not found\n'


UnexpectedResponse: Unexpected Response: 404 (Not Found)
Raw response content:
b'404 page not found\n'

In [16]:
collection_name = "LLM_V1"
query = "After we walk, my dog is always itchy"
response, file_rank = generate_response(collection_name, query)
print_markdown(response.content)


print("files used: \n")
for file in file_rank:
    print(f"{file}, {file_rank[file]} %")

UnexpectedResponse: Unexpected Response: 404 (Not Found)
Raw response content:
b'404 page not found\n'

In [18]:
from flask import Flask, request, jsonify
from flask_cors import CORS

collection_name = "LLM_V1"

app = Flask(__name__)
CORS(app)  # This will enable CORS for all routes

@app.route('/query', methods=['POST'])
def query_llm():
    data = request.json
    query = data.get('query')
    if not query:
        return jsonify({'error': 'No query provided'}), 400

    try:
        response, files = generate_response(collection_name, query)
        return jsonify({
            'response': response.content
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.17.96.147:5000
Press CTRL+C to quit


127.0.0.1 - - [03/Dec/2024 10:29:04] "OPTIONS /query HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 10:29:10] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 10:30:03] "OPTIONS /query HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 10:30:08] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 10:31:00] "OPTIONS /query HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 10:31:22] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 10:32:38] "OPTIONS /query HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 10:32:43] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 10:33:27] "OPTIONS /query HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 10:33:29] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 10:33:39] "OPTIONS /query HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 10:33:40] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 10:34:05] "OPTIONS /query HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 10:34:09] "POST /query HTTP/1.1" 200 -
